# Inicjalizacja

In [ ]:
%matplotlib inline

from evaluator import evaluate, calc_impact, create_functions_in_db, calc_prs_bugginess, overwrite_bugginess_function, get_considered_prs
from definitions import Repository
from configuration import ProjectConfiguration
import db
import smells
import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


#plots size in inches
plt.rcParams["figure.figsize"] = (10,5)

In [ ]:
config = ProjectConfiguration()

db.prepare(config.connstr)

dbsession = db.get_session()
create_functions_in_db(dbsession)

# Ewaluacja

In [ ]:
repositories = list(map(lambda repository_name: dbsession.query(Repository).filter(Repository.full_name == repository_name).first(), config.projects))

if None in repositories:
    raise LookupError("One of repositories does not exist in the database")

## Udział smelly prs wśród badanych prs (reprodukcja)

In [ ]:
chosen_simple_tests = [
    smells.lack_of_review,
    smells.missing_description,
    smells.large_changesets,
    smells.sleeping_reviews,
    smells.review_buddies,
    smells.ping_pong
]
chosen_complex_tests = [
    (smells.union, [smells.lack_of_review,
                    smells.missing_description,
                    smells.large_changesets,
                    smells.sleeping_reviews,
                    smells.review_buddies,
                    smells.ping_pong]),
    (smells.intersection, [smells.lack_of_review,
                    smells.missing_description,
                    smells.large_changesets,
                    smells.sleeping_reviews,
                    smells.review_buddies,
                    smells.ping_pong])
]

In [ ]:
smells_evaluations = {}

for repository in repositories:
    tests_results = list(map(lambda simple_test: evaluate(repository.full_name, simple_test), chosen_simple_tests))
    tests_results.extend(list(map(lambda complex_test: evaluate(repository.full_name, complex_test[0], complex_test[1]), chosen_complex_tests)))
    smells_evaluations[repository.full_name] = tests_results

In [ ]:
# display results as a text
print("Smell / repository".ljust(30), end="\t")
column_width=max(len(repo.name) for repo in repositories)+1
print(*map(lambda repo: repo.name.ljust(column_width), repositories),sep="\t")

for i in range(0, len(chosen_simple_tests)):
    print(next(iter(smells_evaluations.values()))[i].evaluator_name.ljust(30), end="\t")
    print(*map(lambda tests_results: f"{round(tests_results[i].percentage*100,2)}%".rjust(column_width), smells_evaluations.values()), sep="\t")

for i in range(len(chosen_simple_tests), len(chosen_simple_tests)+len(chosen_complex_tests)):
    print(next(iter(smells_evaluations.values()))[i].evaluator_name.ljust(30), end="\t")
    print(*map(lambda tests_results: f"{round(tests_results[i].percentage*100,2)}%".rjust(column_width), smells_evaluations.values()), sep="\t")

In [ ]:
# display results as a plot
labels = list(map(lambda eval: eval.evaluator_name, next(iter(smells_evaluations.values()))))
x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
counter = 0.5
for repository in smells_evaluations:
    bar = ax.bar(x - width/2 + width/len(smells_evaluations)*counter, list(map(lambda evaluation: evaluation.percentage, smells_evaluations[repository])), width/len(smells_evaluations), label=repository)
    counter+=1
ax.set_xticks(x, labels, rotation="vertical")
ax.set_ylim([0, 1])
ax.legend()
fig.tight_layout()
plt.show()

## Wpływ smelli na prawdopodobieństwo wprowadzenia błędu

In [ ]:
tests_for_impact_evaluation = [
    ("Lack of code review", smells.lack_of_review),
    ("Missing PR description", smells.missing_description),
    ("Large changeset", smells.large_changesets),
    ("Sleeping reviews", smells.sleeping_reviews),
    ("Review Buddies", smells.review_buddies),
    ("Ping-pong reviews", smells.ping_pong)
]

In [ ]:
impact_evaluations = {}

for repository in repositories:
    print(f"[{repository.full_name}] Started impact evaluation")
    impact_evaluations[repository.full_name] = list(map(lambda pair: (pair[0], calc_impact(session = dbsession,
                                                                 repo = dbsession.get(Repository, repository.id),
                                                                 evaluator = pair[1],
                                                                 evaluator_args=None)), tests_for_impact_evaluation))
    print(f"[{repository.full_name}] Finished impact evaluation")

In [ ]:
# display results as text
print(
    f"Percentage of pulls where at least one file was changed next by bug solving PR:")

print("Smell / repository".ljust(30), end="\t")
column_width = max(column_width, 22)
print(*map(lambda repo: repo.name.ljust(column_width), repositories),sep="\t")

for i in range(0, len(tests_for_impact_evaluation)):
    print(next(iter(impact_evaluations.values()))[i][0].ljust(30), end="\t")
    print(*map(lambda tests_results: (f"+{round(tests_results[i][1][0]*100,1)}% ".rjust(7)+
                                     f"-{round(tests_results[i][1][1]*100,1)}% ".rjust(7)+"Δ="+
                                     (('+' if tests_results[i][1][1]>tests_results[i][1][0] else '')+
                                     f"{round((tests_results[i][1][1]-tests_results[i][1][0])*100,1)}%").rjust(6)).rjust(column_width), impact_evaluations.values()), sep="\t")

In [ ]:
# display results as a plot
labels = list(map(lambda eval: eval[0], next(iter(impact_evaluations.values()))))
x = np.arange(len(labels))

width=0.35
fig, ax = plt.subplots()
counter = 0.5
for repository in impact_evaluations:
    bar = ax.bar(x - width/2 + width/len(impact_evaluations)*counter, list(map(lambda evaluation: evaluation[1][1]-evaluation[1][0], impact_evaluations[repository])), width/len(smells_evaluations), label=repository)
    counter+=1
ax.set_xticks(x, labels, rotation="vertical")
ax.set_ylim([-1, 1])
ax.legend()
plt.show()

## Metryki

Funkcja zliczająca metrykę bugginess może zostać częściowo nadpisana, domyślnie jest ona następująca:
$
\left\{\begin{array}{ll}
    f(0)=0 &\\
    f(n)=f(x_{n-1})+\frac{n^{-2}}{\text{number of files in original PR}} &\text{gdy n-ty PR zmieniający plik naprawia defektu}\\
    f(n)=f(n-1) &\text{gdy n-ty PR zmieniający plik nie naprawia defektu}
  \end{array}\right.
$

Można nadpisać środkowe równanie.

Domyślnie $n\in[0; 4]\cap\mathbb{N}$, jednak głębokość przeszukiwania również może zostać nadpisana.

In [ ]:
# bugginess function can be overwritten, below current expression
# overwrite_bugginess_function(dbsession, "res + pow(i,-2)/number_of_files")

In [ ]:
calculated_metrics = [
    metrics.review_window_metric,
    metrics.review_window_per_line_metric#,
    # metrics.review_chars,
    # metrics.review_chars_code_lines_ratio,
    # metrics.reviewed_lines_per_hour
]

In [ ]:
metrics_evaluations = {}

for repository in repositories:
    print(f"[{repository.full_name}] Started metrics evaluation")
    for metric in calculated_metrics:
        print(f"[{repository.full_name}] Calculating {metric.__name__}")
        metrics_evaluations[repository.full_name] = metrics_evaluations.get(repository.full_name, [])
        metrics_evaluations[repository.full_name].append(evaluate(repository.full_name, metric))
    metrics_evaluations[repository.full_name] = metrics_evaluations.get(repository.full_name, [])
    metrics_evaluations[repository.full_name].append(evaluate(repository.full_name, calc_prs_bugginess))
    # can be changed for depth other than 4 (below no limit)
    # metrics_evaluations[repository.full_name] = metrics_evaluations.get(repository.full_name, [])
    # metrics_evaluations[repository.full_name].append(calc_prs_bugginess(repository, get_considered_prs(repository,db.get_session()), depth=None))
    print(f"[{repository.full_name}] Finished metrics evaluation")

# create dataframes
metrics_evaluations_df = {}
for repository in repositories:
    df = pd.DataFrame()
    for m in metrics_evaluations[repository.full_name]:
        df[m.metric_name] = m.to_list(dbsession)
    metrics_evaluations_df[repository.full_name] = df

overall_metrics_df = pd.concat(metrics_evaluations_df.values())

In [ ]:
# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# ax.boxplot(list(map(lambda m: m.to_list(dbsession), metrics_evaluations)),
#     showfliers=False, notch=True, vert=False)
# plt.show()

In [ ]:
# calculate correlation
correlations = {}
for repository in repositories:
    correlations[repository.full_name] = metrics_evaluations_df[repository.full_name].corr()
overall_correlations = overall_metrics_df.corr()

In [ ]:
# display correlation matrix
for repository in repositories:
    print(f"Correlation for repository {repository.full_name}")
    print(correlations[repository.full_name])
    print()
print(f"Correlation for all repositories")
print(overall_correlations)

In [ ]:
# display correlation heatmaps
for repository in repositories:
    hm = sns.heatmap(correlations[repository.full_name], annot = True)
    hm.set(title = f"Correlation matrix of reviews metrics and bugginess metrics for {repository.full_name}\n")
    plt.show()
hm = sns.heatmap(overall_correlations, annot = True)
hm.set(title = f"Correlation matrix of reviews metrics and bugginess metrics for all repositories\n")
plt.show()

In [ ]:
# display plots of bugginess against each metrics
# overall_metrics_df = pd.concat(metrics_evaluations_df.values())
# upper_limit = overall_metrics_df.quantile(0.95)
# fig, axs = plt.subplots(len(df.columns)-1)
#
# index = 0
# for column_name in overall_metrics_df.columns[:-1]:
#     col = getattr(overall_metrics_df, column_name)
#     for repository in repositories:
#         df = metrics_evaluations_df[repository.full_name]
#         xs = df[(col < upper_limit[column_name]) & (df.bugginess < upper_limit["bugginess"])][column_name]
#         ys = df[(col < upper_limit[column_name]) & (df.bugginess < upper_limit["bugginess"])]["bugginess"]
#         axs[index].plot(xs, ys, "ro")
#     axs[index].set_xlabel(f"{column_name}\nommited {len(df)-len(xs)} of {len(df)} prs")
#     axs[index].set_ylabel("bugginess")
#     index += 1
#
# plt.subplots_adjust(top=1, bottom=0, hspace=0.25, wspace=1)
# fig.set_figheight((len(df.columns)-1)*fig.get_figheight())
# plt.show()